In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
#from drugbank_downloader import download_drugbank, open_drugbank
import repos_tools
import proximity
import pandas as pd
import matplotlib.pyplot as plt
import attila_utils
from toolbox import wrappers
from toolbox import network_utilities
import pickle

GenRev not found, steiner wont work
Import error: Negex. Using keyword matching instead
Import error: Funcassociate. Make sure that funcassociate is in toolbox!


## Running interactively

In [2]:
fpath = '../../results/2021-08-11-drugbank/drugbank-filtered-proteins.csv'
drugbank_prot = pd.read_csv(fpath, index_col=(0, 1), dtype={'entrez_id': 'str'})
drugbank_prot

symbol  \
drugbank_id uniprot_id          
DB00006     P00734         F2   
DB00007     P30968      GNRHR   
DB00014     P22888      LHCGR   
            P30968      GNRHR   
DB00035     P30518      AVPR2   
...                       ...   
DB15822     P21802      FGFR2   
DB15873     P30874      SSTR2   
DB15982     P03952      KLKB1   
DB16019     P07288       KLK3   
DB16353     P23467      PTPRB   

                                                                   name  \
drugbank_id uniprot_id                                                    
DB00006     P00734                                          Prothrombin   
DB00007     P30968              Gonadotropin-releasing hormone receptor   
DB00014     P22888         Lutropin-choriogonadotropic hormone receptor   
            P30968              Gonadotropin-releasing hormone receptor   
DB00035     P30518                              Vasopressin V2 receptor   
...                                                                 ...   
DB15822     P21802                  Fibroblast growth factor receptor 2   
DB15873     P30874                         Somatostatin receptor type 2   
DB15982     P03952                                    Plasma kallikrein   
DB16019     P07288                            Prostate-specific antigen   
DB16353     P23467      Receptor-type tyrosine-protein phosphatase beta   

                       category organism known_action    actions     hgnc_id  \
drugbank_id uniprot_id                                                         
DB00006     P00734       target   Humans          yes  inhibitor   HGNC:3535   
DB00007     P30968       target   Humans          yes    agonist   HGNC:4421   
DB00014     P22888       target   Humans          yes    agonist   HGNC:6585   
            P30968       target   Humans          yes    agonist   HGNC:4421   
DB00035     P30518       target   Humans          yes    agonist    HGNC:897   
...                         ...      ...          ...        ...         ...   
DB15822     P21802       target   Humans      unknown  inhibitor   HGNC:3689   
DB15873     P30874       target   Humans          yes     binder  HGNC:11331   
DB15982     P03952       target   Humans          yes  inhibitor   HGNC:6371   
DB16019     P07288       target   Humans          yes     binder   HGNC:6364   
DB16353     P23467       target   Humans          yes  inhibitor   HGNC:9665   

                                  group entrez_id  
drugbank_id uniprot_id                             
DB00006     P00734             approved      2147  
DB00007     P30968             approved      2798  
DB00014     P22888             approved      3973  
            P30968             approved      2798  
DB00035     P30518             approved       554  
...                                 ...       ...  
DB15822     P21802             approved      2263  
DB15873     P30874             approved      6752  
DB15982     P03952             approved      3818  
DB16019     P07288             approved       354  
DB16353     P23467      investigational      5787  

[12999 rows x 9 columns]

First we calculate proximities for five drugs asynchronously given the knowledge based AD gene set.

In [3]:
drugbank_prot_small = drugbank_prot.iloc[0:9]
val = proximity.calculate_proximities(drugbank_prot_small, asynchronous=True)

{'MT-ND2', 'MT-ND1'}
DB00006 processed in 15.7s
DB00007 processed in 19.7s
DB00050 processed in 25.7s
DB00014 processed in 25.8s
DB00035 processed in 41.5s
dumping results to /tmp/2021-09-28T13:35:31.212981.p
total runtime: 43.1s


Next we repeat the calculations but **synchronously**.  We see that although individual drugs are computed faster the total runtime is still much slower.

In [4]:
val1 = proximity.calculate_proximities(drugbank_prot_small, asynchronous=False)

{'MT-ND2', 'MT-ND1'}
DB00006 processed in 9.9s
DB00007 processed in 14.9s
DB00014 processed in 18.5s
DB00035 processed in 29.2s
DB00050 processed in 18.1s
dumping results to /tmp/2021-09-28T13:37:03.566867.p
total runtime: 92.3s


Now repeat asynchronously but with the **knowledge, TWAS2+, IAPS** AD gene set.  Larger disease gene set increases run times.

In [5]:
val = proximity.calculate_proximities(drugbank_prot_small, asynchronous=True, dis_genes_fpath='../../results/2021-07-01-high-conf-ADgenes/AD-genes-knowledge-TWAS2plus-IAPS')

{'MT-ND2', 'MT-ND1'}
DB00006 processed in 34.3s
DB00007 processed in 50.1s
DB00050 processed in 69.6s
DB00014 processed in 76.8s
DB00035 processed in 107.7s
dumping results to /tmp/2021-09-28T13:38:52.607346.p
total runtime: 109.0s


In [6]:
val

,name,type,groups,symbol,hgnc_id,d,avg_d_H0,sdev_d_H0,z,p
drugbank_id,,,,,,,,,,
DB00006,Bivalirudin,small molecule,approved|investigational,F2,HGNC:3535,2.0,1.522000,0.538067,0.888365,0.812828
DB00007,Leuprolide,small molecule,approved|investigational,GNRHR,HGNC:4421,2.0,2.012000,0.379284,-0.031639,0.487380
DB00014,Goserelin,small molecule,approved,LHCGR|GNRHR,HGNC:6585|HGNC:4421,2.0,1.918500,0.274058,0.297382,0.616913
DB00035,Desmopressin,small molecule,approved,AVPR2|AVPR1A|AVPR1B,HGNC:897|HGNC:895|HGNC:896,2.0,1.948667,0.236240,0.217293,0.586010
DB00050,Cetrorelix,small molecule,approved|investigational,GNRHR|LHCGR,HGNC:4421|HGNC:6585,2.0,1.918500,0.274058,0.297382,0.616913


## Running from the script

The `knowledge-test.cfg` config file contains all the conditions.

In [7]:
%%bash
cd ../../results/2021-09-25-parallel/
cat knowledge-test.cfg

[DEFAULT]
dis_genes_fpath = /home/attila/CTNS/results/2021-07-01-high-conf-ADgenes/AD-genes-knowledge
network_fpath = /home/attila/CTNS/resources/PPI/Cheng2019/network.sif
id_mapping_file = /home/attila/CTNS/resources/PPI/geneid_to_symbol.txt
drugbank_prot_fpath = /home/attila/CTNS/results/2021-08-11-drugbank/drugbank-filtered-proteins.csv
drugbank_all_drugs_fpath = /home/attila/CTNS/results/2021-08-11-drugbank/drugbank-all-drugs.csv
asynchronous = yes
max_workers = 7
test_run = yes
out_csv = knowledge-test.csv


The config file is passed to `proximity.py`

In [11]:
%%bash
cd ../../results/2021-09-25-parallel/
/home/attila/CTNS/notebooks/src/proximity.py knowledge-test.cfg

GenRev not found, steiner wont work
Import error: Negex. Using keyword matching instead
Import error: Funcassociate. Make sure that funcassociate is in toolbox!
{'MT-ND1', 'MT-ND2'}
DB00007 processed in 14.0s
DB00050 processed in 23.6s
DB00006 processed in 14.8s
DB00035 processed in 28.6s
DB00014 processed in 18.8s
dumping results to knowledge-test.csv.p
total runtime: 30.0s
Results written to knowledge-test.csv


In [12]:
%connect_info

{
  "shell_port": 35363,
  "iopub_port": 45771,
  "stdin_port": 40123,
  "control_port": 33469,
  "hb_port": 36025,
  "ip": "127.0.0.1",
  "key": "305f64ea-bdbc2fc73fddd327665861c4",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-33f032d5-6036-45d4-be0c-690eeada6b8b.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
